In [ ]:
%%bash
python3 -m pip install --user cookiecutter

In [ ]:
#export PATH=$HOME/.local/bin:$PATH

In [ ]:
%%bash
/home/jupyter/.local/bin/cookiecutter --no-input -f --output-dir '/home/jupyter/ssw'  https://github.com/terraform-google-modules/terraform-google-module-template.git

In [ ]:
!sudo apt-get install google-cloud-sdk-config-connector

In [ ]:
!gcloud services enable cloudasset.googleapis.com


In [ ]:
!gcloud projects add-iam-policy-binding 'steveswalker-sandbox' \
  --member=serviceAccount:service-542763728582@gcp-sa-cloudasset.iam.gserviceaccount.com \
  --role=roles/servicenetworking.serviceAgent

In [ ]:
!gcloud beta resource-config list-resource-types
BigQueryTable

In [ ]:
!gcloud projects add-iam-policy-binding 'steveswalker-sandbox' \
  --member=serviceAccount:542763728582-compute@developer.gserviceaccount.com \
  --role=roles/servicenetworking.serviceAgent

In [ ]:
!gcloud beta resource-config bulk-export \
#  --resource-types='BigQueryTable' \
  --project='steveswalker-sandbox' \
  --resource-format=terraform



In [ ]:
%%bash
mkdir /home/jupyter/tf

In [ ]:
!gcloud beta resource-config bulk-export --project='steveswalker-sandbox' --path=/home/jupyter/tf --resource-format=terraform 

In [ ]:
!gcloud beta notebooks environments list --location=us-central1-a

In [ ]:
!gcloud run services describe --region='us-central1' 'gcs-load-bq'

In [ ]:
!gcloud functions regions list --gen2 

In [ ]:
!gcloud compute images list \
    --project deeplearning-platform-release | grep notebooks

In [ ]:
%%bash
wget https://releases.hashicorp.com/terraform/1.3.6/terraform_1.3.6_linux_amd64.zip
unzip terraform*.zip
rm terraform_1.3.6_linux_amd64.zip

In [ ]:

from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set model_id to the ID of the model to fetch.
dataset_id = 'ds_edw'

# Use the delete_contents parameter to delete a dataset and its contents.
# Use the not_found_ok parameter to not receive an error if the dataset has already been deleted.
client.delete_dataset(
    dataset_id, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(dataset_id))

In [ ]:
blah = !curl \
    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
    -H "Content-Type: application/json" \
    "https://config.googleapis.com/v1alpha2/projects/g-sql-morphic-luminous/locations/us-central1/deployments"
type(blah)

In [ ]:
# [START functions_cloudevent_storage]
import functions_framework


# Triggered by a change in a storage bucket
@functions_framework.cloud_event
def hello_gcs(cloud_event):
    data = cloud_event.data

    from google.cloud import bigquery

    client = bigquery.Client()
    table_id = "g-sql-morphic-luminous.A_1.taxi_parquet"
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.PARQUET,
    )
    uri = "gs://solution-cdc-bucket/" + data["name"]

    load_job = client.load_table_from_uri(uri, table_id, job_config=job_config)  

    load_job.result()

    destination_table = client.get_table(table_id)
    print("Loaded {} rows.".format(destination_table.num_rows))

In [ ]:
gcloud functions deploy python-finalize-function \
--gen2 \
--runtime=python310 \
--region=REGION \
--source=. \
--entry-point=hello_gcs \
--trigger-event-filters="type=google.cloud.storage.object.v1.finalized" \
--trigger-event-filters="bucket=YOUR_BUCKET_NAME"

In [ ]:
!gcloud auth list

In [ ]:
%%bash
rm function.zip
zip function.zip main.py
zip function.zip requirements.txt
gsutil cp function.zip  gs://solution-cdc-assets-2023